In [3]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_redis import RedisVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
load_dotenv()


True

In [4]:

# Load documents
loader = TextLoader("data/sample_data.txt")
documents = loader.load()

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
api_key = os.getenv("GOOGLE_API_KEY")

# Initialize Google embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Initialize Redis vector store with HNSW index
vector_store = RedisVectorStore(
    redis_url=os.getenv("REDIS_URL"),
    index_name="rag_index",
    embeddings=embeddings,
    index_type="HNSW"  # High-performance vector index
)

# Clear existing index (optional)
vector_store.delete()

# Add documents
vector_store.add_documents(texts)
print("Documents indexed successfully in Redis!")

Documents indexed successfully in Redis!


In [8]:
vector_store.similarity_search("Tanggal Lahir Mario?", k=1)

[Document(metadata={}, page_content='Mario Aprilnino Prasetyo\nSemarang, Indonesia | +62 812 4743 0546 | https://www.linkedin.com/in/mario-aprilnino/ | mario.aprilnino27@gmail.com | https://marioapn.my.id/\n\nEducation\n\nDian Nuswantoro University - Semarang Indonesia\tSep 2022 - Jan 2026 (Expected)\n\nBachelor of Degree in Informatics Engineering, 3.95/4.00')]

In [9]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [11]:
template = """
Anda adalah seorang sistem yang berfungsi 
sebagai orang yang memahami tentang CV Mario

Informasi yang diberikan adalah sebagai berikut:
{context}

Pertanyaan yang diberikan adalah:
{input}
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

chain = (
    # message
    {
        "context": lambda x: vector_store.similarity_search(x["input"], k=2),
        "input": lambda x: x["input"]
    }
    # chart prompt
    | prompt
    # llm
    | llm
)

In [12]:
# Test the pipeline
query = "Siapa Mario, jelaskan tentang Mario"
response = chain.invoke({"input": query})
print(f"Query: {query}\nResponse: {response.content}")

Query: Siapa Mario, jelaskan tentang Mario
Response: Mario Aprilnino Prasetyo adalah seorang mahasiswa yang berasal dari Semarang, Indonesia. Saat ini, ia sedang menempuh pendidikan S1 Teknik Informatika di Universitas Dian Nuswantoro sejak September 2022 dan diperkirakan lulus pada Januari 2026, dengan Indeks Prestasi Kumulatif (IPK) 3.95/4.00. Selain itu, Mario memiliki pengalaman sebagai Backend Developer, terlihat dari keterlibatannya dalam proyek CrowdFunding Api (Oktober 2024) dan Bengkel Koding VSA (Virtual Smart Assitant) (Oktober 2024). Ia memiliki keahlian dalam teknologi seperti Go, Gin, Gorm, PostgreSQL, dan Midtrans (Payment Gateway). Anda dapat menghubungi Mario melalui nomor telepon +62 812 4743 0546, LinkedIn (https://www.linkedin.com/in/mario-aprilnino/), email mario.aprilnino27@gmail.com, atau mengunjungi situs web pribadinya di https://marioapn.my.id/.



In [13]:
def main():
    print("\nSelamat datang! Silakan ajukan pertanyaan tentang Mario. Ketik 'exit' untuk keluar.")
    while True:
        query = input("Anda: ")
        if query.lower() == 'exit':
            print("Sampai jumpa!")
            break
        response = chain.invoke({"input": query})
        print(f"Bot: {response.content}")

if __name__ == "__main__":
    main() 


Selamat datang! Silakan ajukan pertanyaan tentang Mario. Ketik 'exit' untuk keluar.
Bot: Berdasarkan informasi yang diberikan:

*   Mario sedang menempuh pendidikan S1 (Bachelor of Degree) di Universitas Dian Nuswantoro, Semarang.
*   Ia mulai kuliah pada September 2022 dan diperkirakan lulus pada Januari 2026.

Karena tidak ada informasi langsung mengenai tanggal lahir Mario, kita tidak bisa menghitung umur pastinya. Namun, kita bisa memperkirakan:

*   Jika dia mulai kuliah di usia 18 tahun (usia umum untuk mahasiswa baru), maka saat ini (pertengahan 2024) kemungkinan usianya sekitar 20 tahun.

Bot: Mario tinggal di Semarang, Indonesia.

Bot: Mario tinggal di Semarang, Indonesia.



KeyboardInterrupt: Interrupted by user